# Imports

In [ ]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from pandas_profiling import ProfileReport 

%matplotlib inline

from math import ceil
from sklearn.cluster import KMeans
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV, LassoCV, RidgeCV

# Reading the Data

In [ ]:
df = pd.read_csv('donors.csv', header=0, index_col=[0,'CONTROLN'], low_memory=False, 
                 dtype={'TCODE':'str','HPHONE_D':'str','RFA_2F':'str','WEALTH1':'str','WEALTH2':'str','INCOME':'str'})
df.head()

In [ ]:
df.shape

# Initial Analysis

In [ ]:
# Checking for duplicates:
df.duplicated().sum()

In [ ]:
# check df data types:
df.dtypes

In [ ]:
# replace "" by nans:
df.replace("", np.nan, inplace=True)

In [ ]:
# Dealing with the white spaces where they aren´t supposed to be according to the MetaData:
lista = []
for column in df.columns:
    if (df[column].isin([' ']).sum()>0):
        lista.append(column)
# Decide which ones to replace by NaN or not 
for feature in lista:
    print(df[feature].value_counts())        

#variables in which i´ll replace the ' ' by Nan:
to_be_replaced=['OSOURCE','PVASTATE','DOMAIN','HOMEOWNR','GENDER','DATASRCE','RFA_3','RFA_4','RFA_5','RFA_6','RFA_7','RFA_8',
                'RFA_9','RFA_10','RFA_11','RFA_12','RFA_13','RFA_14','RFA_15','RFA_16','RFA_17','RFA_18','RFA_19','RFA_20',
               'RFA_21','RFA_22','RFA_23','RFA_24']

for feature in to_be_replaced:
    df[feature].replace(" ", np.nan, inplace=True)
    
# CHILD12 e assim considerei o ' 'como ausencia de filhos e nao como missing values

# In the variables 'COLLECT1'-'PLATES' the white space corresponds to the category 'N':
for feature in df.columns[52:70]:
    df[feature].replace(" ", 'N', inplace=True)

In [ ]:
# sorted count of missing values:
df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending=False)

In [ ]:
# checking descriptive statistics:
df.describe().T

### Feature Engineering

In [ ]:
df["GENDER"] = df["GENDER"].apply(lambda x: "U" if (x != "F" and x != "M") else x).value_counts()

In [ ]:
df["ODATEDW"] = df["ODATEDW"].apply(lambda x: x[:7])

In [ ]:
df['DOB'] = df['DOB'].apply(lambda x: int(str(x)[:4]) if (str(x)!='nan') else None)
df['DOB'] = 2020 - df['DOB']
df.rename(columns={'DOB':'Age'},inplace=True)

## Missing Values

In [ ]:
# Plotting the distribution of missing values(greater than 10%)
ax = sns.histplot(df.isnull().mean()*100, bins='auto')
ax.set_title('Distribution of Missing values (%) ')
ax.set(xlabel='% of missing values', ylabel='Count of features',xlim=(10,100),ylim=(0,30))
# Colloring red the bars that represent the count of features with % of missing values >= 50% :
for i in range(5,10):
    ax.patches[i].set_color('r')
    ax.patches[i].set_linestyle(':')
plt.show(ax)

### Removing the features that have missing values

We are removing all the columns that have more than 50% of Missing Values

### Nota: Ver as ADATES

In [ ]:
cols = df.columns[df.isnull().mean() > 0.5]

display(pd.DataFrame(df[cols].isnull().mean()*100, columns=['% Missing Values']).sort_values(by='% Missing Values',ascending=False))

df = df.drop(cols, axis=1)

#### Metric and Non-Metric Variables:

In [ ]:
def update_metric_feat(df):
    metric_features = df.select_dtypes(include=np.number).columns
    non_metric_features = df.select_dtypes(exclude=np.number).columns
    return metric_features.tolist(),non_metric_features.tolist()

metric_features = update_metric_feat(df)[0]
non_metric_features = update_metric_feat(df)[1]

## Outliers

#### Box Plots:

In [ ]:
# All Numeric Variables' Box Plots in one figure
sns.set()

# Prepare figure. Create individual axes where each box plot will be placed
fig, axes = plt.subplots(52, ceil(len(metric_features) / 52), figsize=(50, 50), squeeze=True)

# Plot data
# Iterate across axes objects and associate each box plot:
for ax, feat in zip(axes.flatten(), metric_features): 
    sns.boxplot(x=df[feat], ax=ax)
#     ax.set_title(feat, y=-0.21)

# Adjusting my subplots:
fig.tight_layout()

# Layout
# Add a centered title to the figure:
title = "Numeric Variables' Box Plots"
plt.suptitle(title)

plt.show()

Steps:
- 1)Impute the missing values with the median for the metric features and mode for the non metric, because those measures are robust to outliers
- 2)Drop outliers using the Local Outlier Factor method or/and its intersection with the methods IQR and z-score
- 3)Now that we don´t have outliers anymore we can use a better method for the missing data imputation
- 4)After the feature selection we can go back and only drop the outliers on the variables that we kept

Creating an auxiliary df ( so that later I can use another method for the data imputation), and Filling the Categorical Variables with the mode and the Numeric Variables with the Median

In [ ]:
aux_df = df.copy()

for column in aux_df[metric_features]:
    aux_df[column] = aux_df[column].fillna(aux_df[column].median())

Now, I can drop the outliers on our original df, based on our aux_df without missing values:

In [ ]:
# Dealing with outliers on the metric features:
from sklearn.neighbors import LocalOutlierFactor

# we are only finding out outliers among the metric features:
aux_df = aux_df[metric_features]
# identifying outliers in our dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(aux_df.values)

# select all rows that are not outliers
mask = yhat != -1

# variable that contains the original number of observations:
numero_obs_original = df.shape[0]
df = df.drop(index=aux_df.iloc[~mask, :].index)


print('% eliminated=',(1-len(df)/numero_obs_original)*100,'%')

Now our df is free of outliers so we can make a better imputation of the missing values in the metric features

#### Later when we have done all of our feature selection, we can go back here and apply the following methods:

In [ ]:
def z_score_outlier_detection(df,column):
    mean = df[column].mean()
    std = df[column].std()
    threshold = 3
    z_s = (df[column]-mean)/std
    return z_s.apply(lambda z: z>threshold) # true if outlier

In [ ]:
# Defining a function that removes the outliers from the metric features, using the IQR method:
def iqr_outlier_detection(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = (q3 - q1) 
    lower_lim = q1 - 5 * iqr
    upper_lim = q3 + 5 * iqr
    return df[column].apply(lambda x: (x < lower_lim or x > upper_lim))   # true if outlier

In [ ]:
# aux_df = df.copy()

# aux_df['outlier_z_score?']=False
# aux_df['outlier_iqr?']=False

# for feature in metric_features:
#     aux_df['outlier_z_score?'] = aux_df['outlier_z_score?']|z_score_outlier_detection(aux_df,feature)
#     aux_df['outlier_iqr?'] = aux_df['outlier_iqr?']|iqr_outlier_detection(aux_df,feature)

# teste =aux_df['outlier_z_score?'] & aux_df['outlier_iqr?']
# print('% to be eliminated=',(teste.sum()/95412)*100,'%')

## Fill missing values (Data imputation)

Filling the Categorical Variables with the mode and the Numeric Variables with the KNN imputer

In [ ]:
# Non Metric features:
for column in df[non_metric_features]:
    df[column] = df[column].fillna(df[column].mode()[0])

# Metric features:
# Creating new df copy to explore neighbordhood imputation
df_neighbors = df.copy()

# Seeing rows with NaNs
nans_index = df_neighbors.isna().any(axis=1)
df_neighbors[nans_index]

# KNNImputer - only works for numerical varaibles
imputer = KNNImputer(n_neighbors=5, weights="uniform")
df_neighbors[metric_features] = imputer.fit_transform(df_neighbors[metric_features])

# See rows with NaNs imputed
df_neighbors.loc[nans_index, metric_features]

# let's keep the df_neighbors dataframe
df = df_neighbors.copy()

In [ ]:
df.isnull().sum()

# Metric Features

## Low Variance

Removing variables with low Variance

Done on a scaled data using MinMax Scaler

If the variance is too low, it means that it does not change much and hence it can be ignored


#### -> Function that plots Histograms of a set of variables:

In [ ]:
def histograms(df,features, bins='auto',figsize=(15, 8)):
    # Put this Variables' Histograms in one figure
    sns.set()

    # Prepare figure. Create individual axes where each histogram will be placed
    fig, axes = plt.subplots(ceil(len(features) / 4),4, figsize=figsize)

    # Plot data
    # Iterate across axes objects and associate each histogram:
    for ax, feat in zip(axes.flatten(), features):
        ax.hist(df[feat], bins=bins)
        ax.set_title(feat, y=-0.17)

    # Layout
    # Add a centered title to the figure:
    title = "Variables' Histograms"
    plt.suptitle(title)
    plt.show()

In [ ]:
# Scalling our data:
scaler = MinMaxScaler()
# transform data:
scaled = scaler.fit_transform(df[metric_features])
scaled = pd.DataFrame(scaled,columns = metric_features)

In [ ]:
low_variance_features = []
for feature in metric_features:
    if (scaled[feature].var())<0.0009:
        low_variance_features.append(feature)

print('The following are redundant variables because of their low variance:\n',low_variance_features)

#Plotting an histogram of those variables:
histograms(df,low_variance_features, bins='auto',figsize=(35, 50))

# Dropping (note: I wont drop the boolean variable HPHONE_D because it only takes 0 and 1 values so it will easily have a std 
# value above my threshold, but I won´t consider it has being redundant):
df.drop(columns=low_variance_features, inplace=True)

#updating metric features list:
metric_features = update_metric_feat(df)[0]



In [ ]:
display(pd.DataFrame(scaled[metric_features].var(), columns=['Variance']).sort_values(by='Variance'))

## Correlation
Remove the features that have high correlation values (threshold of 70%) -> To avoid multicolinearity

For each pair of correlated features, our method identifies one of the features for removal (because we only need to remove one of them)

Make visualization with the high correlations (ex:heatmap that shows all the features that have at least one correlation above the threshold)

Make list of correlated features that will be removed, or show the highly correlated pairs of features in a dataframe

Features with high correlation are more linearly dependent and hence have almost the same effect on the dependent variable. So, when two features have high correlation, we can drop one of the two features

#### -> Function that plots an heatmap of the correlation matrix passed as input

In [ ]:
def corr_heatmap(corr):    
    # Prepare figure
    fig = plt.figure(figsize=(50, 30))
    # Build annotation matrix (values above |0.7| will appear annotated in the plot)
    mask_annot = np.absolute(corr.values) >= 0.7
    annot = np.where(mask_annot, corr.values, np.full(corr.shape,""))
    # Plot heatmap of the correlation matrix
    sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
                fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5, annot_kws={"size": 10})
    # Layout
    fig.subplots_adjust(top=0.95)
    fig.suptitle("Correlation Matrix", fontsize=20)
    plt.show()

#### -> Handling with highly correlated Features:

In [ ]:
# Creating correlation matrix with the absolute values of the corr:
corr = df.corr().abs()

# Selecting upper triangle of the correlation matrix:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

# Finding the index of the feature's columns with correlation greater than 0.70 (>=very strong correlation)
to_drop = [column for column in upper.columns if any(upper[column] > 0.70)]

print('The features to be removed because of their high correlation are:\n',to_drop)

# Saving the pairs of correlated features so I can plot an heatmap of their correlations:

# I´ll save both the dropped feature and the one that was correlated with it:
records = pd.DataFrame(columns = ['dropped_feat', 'corr_feat'])
for column in to_drop:
    corr_features = list(upper.index[upper[column].abs() > 0.7])
    drop_features = [column for _ in range(len(corr_features))]
    # Record the information (need a temp df for now)
    aux_df = pd.DataFrame.from_dict({'dropped_feat': drop_features,'corr_feat': corr_features})

    # Add to dataframe
    records = records.append(aux_df, ignore_index = True)


corr_matrix_plot = corr.loc[list(set(records['corr_feat'])),list(set(records['dropped_feat']))]
title = "Correlations Above Threshold"
# * the features on the x axis are the ones that are going to be removed


# Plotting an heatmap:
corr_heatmap(np.round(corr_matrix_plot,decimals=2))


# Dropping the highly correlated features 
df.drop(columns = to_drop, inplace=True)

# updating metric features list:
metric_features = update_metric_feat(df)[0]

#### Some explanations

[Handle with variables of central tendency measures]

Once there are variables that provide the same information but in different measures, we decide to keep the variables that refer to the median because it's less sensitive to outliers and remove the others ones in these specific cases.

One example of the situation described above is the variable HHP1 (Median Person Per Household) and HHP2 (Average Person Per Household).


[Handle with redundant variables]

Since there are variables that have repeated information, that is, variables that englobe other variables, let's take into consideration, for example, HHN3 (Percent 3 or More Person Households) and HHN4 (Percent 4 or More Person Households). We can soon conclude that HHN4 is mentioned in HHN3, once 'more than 3' englobe the percent of 4 and so on.

In these particular cases, we decide to keep the variables that provide the same meaning as the others of the same kind, without losing information, and discard the ones that are already indirectly specified in the ones that we chose to keep.

### Scale/Standardize our data

In [ ]:
df_standard = df.copy()
scaler = StandardScaler()
scaled_feat = scaler.fit_transform(df_standard[metric_features])
scaled_feat

In [ ]:
# See what the fit method is doing :
print("Parameters fitted:\n", scaler.mean_, "\n", scaler.var_)

In [ ]:
df_standard[metric_features] = scaled_feat
df_standard.head()

In [ ]:
# Checking mean and variance of standardized variables
df_standard[metric_features].describe().round(2)

In [ ]:
df = df_standard.copy()

### Apply kmeans for feature selection (using it to find the clusters which then are my target variable)

In [ ]:
# Defining the number of clusters:
range_clusters = range(1, 11)
inertia = []
for n_clus in range_clusters:  # iterate over desired ncluster range
    kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=15, random_state=1)
    kmclust.fit(df[metric_features])
    inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution
    
# The inertia plot
plt.figure(figsize=(9,5))
plt.plot(inertia)
plt.ylabel("Inertia: SSw")
plt.xlabel("Number of clusters")
plt.title("Inertia plot over clusters", size=15)
plt.show()

In [ ]:
# final cluster solution
number_clusters = 3
kmclust = KMeans(n_clusters=number_clusters, init='k-means++', n_init=15, random_state=0)
km_labels = kmclust.fit_predict(df[metric_features])
df_concat = df.copy() 
df_concat['labels']=km_labels
df_concat

In [ ]:
# Lasso regression
# separating the income variable from the independent numeric variables:
data = df_concat[metric_features]
target = df_concat.iloc[:,-1]

reg = LassoCV()
reg.fit(X = data, y = target)

# The alpha value is associated to the regularization strength. Regularization improves the conditioning of the problem and 
# reduces the variance of the estimates. Larger values specify stronger regularization:
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)

# The score return the coefficient of determination R^2 of the prediction:
print("Best score using built-in LassoCV: %f" %reg.score(data,target))

# coef will have as index the data columns names and the coef_ attribute as value:
coef = pd.Series(reg.coef_, index = data.columns)
coef

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
coef.sort_values()

In [ ]:
def plot_importance(coef,name):
    imp_coef = coef.sort_values()
    plt.figure(figsize=(55,55))
    imp_coef.plot(kind = "barh")
    plt.title("Feature importance using " + name + " Model")
    
    plt.show()

plot_importance(coef, "Lasso")

In [ ]:
# # Ridge classifier
# ridge = RidgeClassifierCV().fit(X = data,y = target)
# coef_ridge = pd.Series(ridge.coef_[0], index = data.columns)
# plot_importance(coef_ridge,'RidgeClassifier')

# ridge.score(data,target)

In [ ]:
# Dropping the variables according to the Lasso Regression:
to_drop = np.array(metric_features)[(coef ==0).values]

print('The following variables were dropped because according to the Lasso Regression they weren´t important to determine the \
cluster:', to_drop)

df.drop(columns=to_drop, inplace=True)

# updating metric features list:
metric_features = update_metric_feat(df)[0]

## Dimensionality Reduction

### PCA

In [ ]:
df_pca = df.copy()

# Use PCA to reduce dimensionality of data
pca = PCA()
pca_feat = pca.fit_transform(df_pca[metric_features])
# pca_feat  

var_explained = pca.explained_variance_ratio_.cumsum()

pca.explained_variance_ratio_[:10].sum()

var_explained = var_explained*100 #percentage

fig, ax = plt.subplots(figsize=(8,8))
plt.bar(range(len(var_explained)), var_explained, label='Cumulative _Sum_of_Explained _Varaince', color = 'b',alpha=0.5)
plt.title("Around 80% of variance is explained by the first 40 principal components ");

In [ ]:
print(pca_feat.shape)
print(df_pca[metric_features].shape)
pd.DataFrame(pca_feat)

In [ ]:
cov_matrix = df_pca[metric_features].cov()
cov_matrix

In [ ]:
# Composing back the covariance matrix
pd.DataFrame(pca.components_.T @ np.diag(pca.explained_variance_) @ pca.components_, 
             index=cov_matrix.index, 
             columns=cov_matrix.columns)

In [ ]:
# Obtaining the projected observations on the principal components axes (linear combinations)
pd.DataFrame(df_pca[metric_features].values @ pca.components_.T, 
             index=df_pca.index,
             columns=[f"PC{i}" for i in range(pca.n_components_)])

In [ ]:
# Output PCA table
pd.DataFrame(
    {"Eigenvalue": pca.explained_variance_,
     "Difference": np.insert(np.diff(pca.explained_variance_), 0, 0),
     "Proportion": pca.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca.explained_variance_ratio_)},
    index=range(1, pca.n_components_ + 1)
).head(41)
# rule(pearson), de manter os que têm um eigenvalue>1 ->>>> manter 23 PC
# na kaiser´s ruler queremos reter 80% da variancia, olhar para a comulative ->>>> manter 40 PC

In [ ]:
# figure and axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(55, 25))

# draw plots
ax1.plot(pca.explained_variance_, marker=".", markersize=12)
ax2.plot(pca.explained_variance_ratio_, marker=".", markersize=12, label="Proportion")
ax2.plot(np.cumsum(pca.explained_variance_ratio_), marker=".", markersize=12, linestyle="--", label="Cumulative")

# customizations
ax2.legend()
ax1.set_title("Scree Plot", fontsize=14)
ax2.set_title("Variance Explained", fontsize=14)
ax1.set_ylabel("Eigenvalue")
ax2.set_ylabel("Proportion")
ax1.set_xlabel("Components")
ax2.set_xlabel("Components")
ax1.set_xticks(range(0, pca.n_components_, 2))
ax1.set_xticklabels(range(1, pca.n_components_ + 1, 2))
ax2.set_xticks(range(0, pca.n_components_, 2))
ax2.set_xticklabels(range(1, pca.n_components_ + 1, 2))

plt.show()

ESCOLHER OS PRINCIPAL COMPONENTS A RETER E FAZER OS CODIGOS SEGUINTES:

In [ ]:
# # Perform PCA again with the number of principal components you want to retain
# pca = PCA(n_components=23)
# pca_feat = pca.fit_transform(df_pca[metric_features])
# pca_feat_names = [f"PC{i}" for i in range(pca.n_components_)]
# pca_df = pd.DataFrame(pca_feat, index=df_pca.index, columns=pca_feat_names)  # remember index=df_pca.index
# pca_df

In [ ]:
# # Reassigning df to contain pca variables
# df_pca = pd.concat([df_pca, pca_df], axis=1)
# df_pca.head()

# # we should use the components for our analysis instead of the numeric variables

INTERPRETATION OF THE PRINCIPAL COMPONENTS AND LOADINGS:

# Categorical Features

#### -> Pieplots

In [ ]:
sns.set()

# Prepare figure. Create individual axes where each histogram will be placed
fig, axes = plt.subplots(23, 2, figsize=(20, 80))

# Plot data
# Iterate across axes objects and associate each histogram:
for ax, feat in zip(axes.flatten(), non_metric_features): # Notice the zip() function and flatten() method
    ax.pie(df[feat].value_counts().values,labels=df[feat].value_counts().index,autopct='%1.1f%%', shadow=True, radius=2)
    ax.set_title(feat)
fig.tight_layout()
# Layout
# Add a centered title to the figure:
title = " Non metric features Pieplots"
plt.suptitle(title)
plt.show()

#### -> Single Value Features

In [ ]:
# get number of unique values for each column using numpy
counts = list(df.nunique())
names = list(df.nunique().index)

def create_dict(keys, values): 
    return dict(zip(keys, values + [None] * (len(keys) - len(values))))
columns = create_dict(names, counts)

# record columns to delete
to_del = dict((k, v) for k, v in columns.items() if v == 1) 
to_del

#### -> Remove the variables that for one category have more than 85% of the observations:

In [ ]:
to_drop=[]
for feature in non_metric_features:
    for proportion in df[feature].value_counts()/len(df):
        if proportion > 0.85:
            to_drop.append(feature)
            break

print('The non metric features that have more than 85% in one of their categories/unique values (to be dropped because \
of their redundancy) are the following:\n',to_drop)

df.drop(columns=to_drop, inplace=True)

# updating non metric features list:
non_metric_features = update_metric_feat(df)[1]

#### ->Drop the categorical features with high variability , in case none of its categories have more than 15%:

In [ ]:
to_drop=[]
for feature in non_metric_features:
    aux =[]
    for proportion in df[feature].value_counts()/len(df):
        if proportion < 0.15:
            aux.append(proportion)
    if len(aux)==len(df[feature].value_counts()): # if all categories have less than 15%;
        to_drop.append(feature)

print('The non metric features that have less than 15% in all of their categories/unique values (to be dropped because \
of their redundancy) are the following:\n',to_drop)

df.drop(columns=to_drop, inplace=True)

# updating non metric features list:
non_metric_features = update_metric_feat(df)[1]

### Reduce Cardinality of the Categorical features!!!!!!!!!!!!!!!!

In [ ]:
# plot with the cardinality of each variable

df[non_metric_features].nunique().plot.bar(figsize=(12,6))
plt.ylabel('Number of unique categories')
plt.xlabel('Variables')
plt.title('Cardinality')

### MCA

### K-Modes and feature importance